<a href="https://colab.research.google.com/github/Firojpaudel/Machine-Learning-Notes/blob/main/Practical%20Deep%20Learning%20For%20Coders/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiLabel Classification And Regression


In the last chapter, we learned some practical tips for training models, like choosing the right learning rates and number of epochs which were really important for good results. Now, this chapter dives into two new types of computer vision problems: multi-label classification (predicting multiple labels for an image or none at all) and regression (predicting numbers instead of categories). Along the way, we'll explore output activations, targets, and loss functions in more depth.

## MultiLabel Classification

*MultiLabel Classification* refers to the problem of identifying the objects in images that may not contain exactly one type of object.

There may be more than one type of object or there may be none.

In previous bear classifier we built, it had no ability to predict "not bear at all". So, this time we are trying to fix that.

First, let's see what a mutilabel dataset looks like. But before that lets set up the notebook.

In [2]:
### Setting up the notebook
##@ Notebook initialization
%reload_ext autoreload
%autoreload 2
%matplotlib inline

##@ Installing dependencies
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

##@ Importing the necessary libraries
from fastbook import *
from fastai.callback.fp16 import *
from fastai.vision.all import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/gdrive


Now getting the data:

We will get the PASCAL datset which has more than one kind of classified object per image.

In [ ]:
##@ Getting the PASCAL dataset
path = untar_data(URLs.PASCAL_2007)

<details>
<summary><b>More about this dataset:</b></summary>
This dataset is a bit different from the ones we've worked with before. Instead of being organized by filenames or folders, it comes with a CSV file that tells us which labels to use for each image.
</details>


In [ ]:
##@ inspecting the dataset
df = pd.read_csv(path/'train.csv')
df.head()

### The Pandas and DataFrames Section in the Book:


In [1]:
##@ We cab access the rows and columns in a dataframe using the .iloc property

df.iloc[:, 0]         #This would return the 1st column from  the dataframe

NameError: name 'df' is not defined